**Midterm Mock Test**

- Name and Student ID:
- Two-hour exam
- Write all your answers on the exam paper. Your answer must be justified clearly
- If you have used python code, you shall convert it into PDF and send it to songqsh2000@yahoo.com
- It is open book. However, any kind of communication during the exam period is not allowed

**Problem 1**

Consider CRR option with the following parameters:

- Model: 2-period CRR for total length 2 year
- Stock initial price: 50
- Volatility: 20 %
- Rate: 5 %
- Put option price: 50
- Put maturity: 2 year

**Questions**

1. Find CRR put price.

In [2]:
# use BSM to calculate CRR option price
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import time

T=2
S0=50
sigma=0.2
r=0.05
K=50
t=0

def d1f(St, K, t, T, r, sigma):
    d1 = (math.log(St / K) + (r + 0.5 * sigma ** 2)
          * (T - t)) / (sigma * math.sqrt(T - t))
    return d1

def BSM_call_value(St, K, t, T, r, sigma):
    d1 = d1f(St, K, t, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    call_value = St * ss.norm.cdf(d1) - math.exp(-r * (T - t)) * K * ss.norm.cdf(d2)
    return call_value

def BSM_put_value_2(St, K, t, T, r, sigma):

    d1 = d1f(St, K, t, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    put_value = math.exp(-r * (T - t)) * K * ss.norm.cdf(-d2) - St * ss.norm.cdf(-d1)
    return put_value

P0=BSM_put_value_2(S0, K, t, T, r, sigma)
print("the CRR put price = ", P0)

the CRR put price =  3.305260764287283


In [4]:
# use CRR model to calculate CRR option price

import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'serif'
from BSM_option_valuation import BSM_call_value

T=2
S0=50
sigma=0.2
r=0.05
K=50
t=0
M=2
otype="put"

def CRR_option_value(S0, K, T, r, sigma, otype, M):
    # Time Parameters
    dt = T / M  # length of time interval
    df = math.exp(-r * dt)  # discount per interval

    # Binomial Parameters
    u = math.exp(sigma * math.sqrt(dt))  # up movement
    d = 1 / u  # down movement
    q = (math.exp(r * dt) - d) / (u - d)  # martingale branch probability

    # Array Initialization for Index Levels
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md

    # Inner Values
    if otype == 'call':
        V = np.maximum(S - K, 0)  # inner values for European call option
    else:
        V = np.maximum(K - S, 0)  # inner values for European put option

    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        V[0:M - z, t] = (q * V[0:M - z, t + 1] +
                         (1 - q) * V[1:M - z + 1, t + 1]) * df
        z += 1
    return V[0, 0]

P0_2 = CRR_option_value(S0, K, T, r, sigma, otype, M)
print("the CRR put price = ", P0_2)

the CRR put price =  2.6625667890451976


**Problem 2**

Suppose a Monte-Carlo method generates a random variable $\hat \alpha$ for the approximation of a deterministic number $\alpha$. Recall that Bias is given by
$$Bias = \mathbb E[\hat \alpha] - \alpha.$$
and the mean square error is given by
$$MSE(\hat \alpha) = \mathbb E[ (\hat \alpha - \alpha)^2 ].$$

**Questions**

1. Prove that 
    $$MSE(\hat \alpha) = |Bias|^2 + Var (\hat \alpha).$$



In [0]:
# 分别展开

**Problem 3**

Consider the following data sets:

Today: Oct 08, 2018

BAC Spot price: 30.27

European Call Maturity: Jun 21, 2019

European Call Strike: 30.0

rate: 0.0264

Market call price: 2.70

**Questions**

1. Compute the BSM implied volatility.

In [14]:
import numpy as np
import pandas as pd
import scipy.optimize as sop
import math
import time
import scipy.stats as ss

S0 = 30.27
r = 0.0264
K=30
call=2.70
today = np.datetime64('2018-10-08')
maturity=np.datetime64('2019-06-21')
T=(maturity-today)/np.timedelta64(1,"D")/365

def d1f(St, K, t, T, r, sigma):
    d1 = (math.log(St / K) + (r + 0.5 * sigma ** 2)
          * (T - t)) / (sigma * math.sqrt(T - t))
    return d1

def BSM_call_value(St, K, t, T, r, sigma):
    d1 = d1f(St, K, t, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    call_value = St * ss.norm.cdf(d1) - math.exp(-r * (T - t)) * K * ss.norm.cdf(d2)
    return call_value

def IVolBsm(S0, K, T, r, P0):
    InitVol = .3
    error = lambda sigma: (BSM_call_value(S0, K, 0, T, r, sigma) - P0)**2
    opt = sop.fmin(error, InitVol);
    return opt[0]

print("implied volatility is:",IVolBsm(S0, K, T, r, call))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 24
implied volatility is: 0.2275195312499998
